In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)

In [3]:
%%time

import sys
sys.path.append('..')
import os
os.nice(19)
%matplotlib inline



import graphlearn.utils.draw as myutils
import graphlearn.discsampler as ds
from eden.converter.graph.gspan import gspan_to_eden
import itertools
import matplotlib.pyplot as plt


# hope this works.,.,. dont want to see warnings
import warnings
warnings.filterwarnings('ignore')


sampler=ds.discsampler()

#sampler.fit(gspan_to_eden( 'bursi.pos.gspan' ),  n_jobs=-1)
#sampler.save('tmp/demo.ge')
#print 'saved'
sampler.load('tmp/demo.ge')
graphs = gspan_to_eden( 'bursi.pos.gspan' )


graphs = itertools.islice(graphs,10)
graphs = sampler.sample( graphs,
               batch_size=1,
               n_jobs=5,
               n_steps=100,
               select_cip_max_tries = 100,
               annealing_factor = 1.0,
               queue_chunk_size = 5,
               radius = 0.15,
               create_n_samples = 30,
               sample_tries = 30
               )
myutils.draw_many_graphs(graphs)


AttributeError: discsampler instance has no attribute 'load'